In [2]:
import pandas as pd
import numpy as np
import random
import json

directorio = pd.read_csv('./WilmerCSV.csv', index_col='ID')

In [3]:
directorio.parent = -1
directorio.head()

,Job Title,Email Address,FirstName LastName,parent
ID,,,,
1,Project Manager,Blake_London9672@supunk.biz,Blake London,-1
2,Assistant Buyer,Camden_Redden9030@bauros.biz,Camden Redden,-1
3,Machine Operator,Lily_Donovan5896@typill.biz,Lily Donovan,-1
4,Pharmacist,Emely_Bingham8992@typill.biz,Emely Bingham,-1
5,Auditor,Bree_Cox8741@sheye.org,Bree Cox,-1


In [4]:

def generar_rama_binaria(adn, degrad):
    nivel = []
    for i in range(len(adn)):
        sub_directorio = directorio.loc[directorio['parent'] == -1]
        prob = random.random() 
        num_ref =2
        
        if prob < degrad[0]:
            nu_ref = 0
        elif prob >= degrad[0] and prob < degrad[0] + degrad[1]:
            num_ref = 1
        
        referidos = sub_directorio.sample(num_ref).index   # **
        directorio.loc[referidos, 'parent'] = adn[i]
        nivel.extend(referidos.values)
    return nivel

In [5]:
# primer paso elegir al azar 3 filas y asignarles el valor 0 a 
# columna parent

adn = 0

sub_directorio = directorio.loc[directorio['parent'] == -1]
referidos = sub_directorio.sample(3).index
directorio.loc[referidos, 'parent'] = adn

for i in range(10):
    if i<=3:
        descendientes = generar_rama_binaria(referidos, [0., 0., 1.])
        
    elif i>3 and i<=6:
        descendientes = generar_rama_binaria(referidos, [0., .25, .75])
        
    else:
        descendientes = generar_rama_binaria(referidos, [.1, .4, .5])

    print('{} \t {}'.format(i+1, len(descendientes)))
    referidos = descendientes.copy()

# esta OK!

1 	 6
2 	 12
3 	 24
4 	 48
5 	 85
6 	 151
7 	 266
8 	 413
9 	 653
10 	 1046


In [5]:
# lista todo el directorio de afiliados ordenado por parent
#directorio.loc[directorio['parent']>-1].sort_values('parent')

In [6]:
directorio.loc[directorio['parent']>-1].count()

Job Title             2707
Email Address         2707
FirstName LastName    2707
parent                2707
dtype: int64

#### seleccionar al azar un elemento del directorio y listar su ascendencia

In [7]:
# seleccionar al azar un elemento del directorio y listar su ascendencia

# seleccion
cliente = directorio.loc[directorio['parent']>-1].sample(1).index

fila = directorio.loc[cliente]
adn = fila['parent'].values[0]
arbol = [fila.index.values[0]]

# busca ascendencia
while True:
    if adn > 0:
        ancestro = directorio.loc[adn]
        arbol.append(adn)
        adn = ancestro['parent']
    else:
        break

# lista ascendencia
print('nivel profundidad: ', len(arbol)-1)
for rama in arbol:
    print('\n', directorio.loc[rama])

nivel profundidad:  10

 Job Title                       IT Support Staff
Email Address         Hazel_Janes4931@bungar.biz
FirstName LastName                   Hazel Janes
parent                                      7300
Name: 8430, dtype: object

 Job Title                                       Operator
Email Address         Kassandra_Sylvester7447@kideod.biz
FirstName LastName                   Kassandra Sylvester
parent                                              4677
Name: 7300, dtype: object

 Job Title                          Stockbroker
Email Address         Tony_Drake809@supunk.biz
FirstName LastName                  Tony Drake
parent                                    7787
Name: 4677, dtype: object

 Job Title                   Service Supervisor
Email Address         Wendy_Owens52@bungar.biz
FirstName LastName                 Wendy Owens
parent                                    1176
Name: 7787, dtype: object

 Job Title                     Machine Operator
Email Address   

### seleccionar al azar un elemento del directorio y listar su descendencia

In [87]:
# seleccionar al azar un elemento del directorio y listar su descendencia
#cliente = directorio.loc[directorio['parent']>-1].sample(1).index

# seleccion
cliente = [1176]

adn = [cliente[0]]
arbol = []

# dic_des { directorio.loc[rama, 'Email Address']: cliente}

# busca descendencia

pata_corta = False 
fin_rep = False

while True:
    
    for ancestro in adn:
        hijos = directorio.loc[directorio['parent'] == ancestro].index.values
        if len(arbol) == 0:
            padre = "null"
        else:
            padre = directorio.loc[ancestro, 'parent']
        
        if len(hijos) > 0:
            arbol.append({"name": ancestro, "parent": padre, "children": list(hijos)})
            
        if len(hijos) < 2: pata_corta = True

    descendientes = directorio.loc[directorio['parent'].isin(adn)].index.values # descendientes debe ser mas preciso 
                                                                                # para establecer rama de descendencia
    if len(descendientes)==0  or fin_rep:
        break
    elif pata_corta:
        fin_rep = True
        

    adn = descendientes.copy()
   


In [74]:
arbol

[{'name': 1176, 'parent': 'null', 'children': [7787, 9433]},
 {'name': 7787, 'parent': 1176, 'children': [2027, 4677]},
 {'name': 9433, 'parent': 1176, 'children': [4258, 7222]},
 {'name': 2027, 'parent': 7787, 'children': [4341]},
 {'name': 4258, 'parent': 9433, 'children': [2054, 2446]},
 {'name': 4677, 'parent': 7787, 'children': [1450, 7300]},
 {'name': 7222, 'parent': 9433, 'children': [2022]},
 {'name': 1450, 'parent': 4677, 'children': [7387]},
 {'name': 2022, 'parent': 7222, 'children': [2479, 8603]},
 {'name': 2054, 'parent': 4258, 'children': [3513, 5173]},
 {'name': 2446, 'parent': 4258, 'children': [3829, 4527]},
 {'name': 4341, 'parent': 2027, 'children': [3510]},
 {'name': 7300, 'parent': 4677, 'children': [1101, 8430]}]

In [88]:
# convierte lista arbol en diccionario anidado para procesarlo
# por visualizador

def agrega_hijo(padre):
    for rama in arbol:
        if rama['name'] == padre:
            dat_niv = {'name': str(rama['name']), 'parent': str(rama['parent'])}
            dat_niv['children'] = [agrega_hijo(children) for children in rama['children']]
            
            if dat_niv['children'] == [None] or dat_niv['children'] == [None, None]:
                dat_niv.pop('children')
                #dat_niv['children'] = [str(children) for children in rama['children']]
            
            return dat_niv

treeData = []

dat_anid = {'name': str(arbol[0]['name']), 'parent': arbol[0]['parent']}

dat_anid['children'] = [agrega_hijo(children) for children in arbol[0]['children']]

treeData.append(dat_anid)



In [85]:
# http://bl.ocks.org/d3noob/8375092 funcion javascript para visualizar
# arbol binario treeData
treeData

[{'name': '1176',
  'parent': 'null',
  'children': [{'name': '7787',
    'parent': '1176',
    'children': [{'name': '2027',
      'parent': '7787',
      'children': [{'name': '4341', 'parent': '2027'}]},
     {'name': '4677',
      'parent': '7787',
      'children': [{'name': '1450', 'parent': '4677'},
       {'name': '7300', 'parent': '4677'}]}]},
   {'name': '9433',
    'parent': '1176',
    'children': [{'name': '4258',
      'parent': '9433',
      'children': [{'name': '2054', 'parent': '4258'},
       {'name': '2446', 'parent': '4258'}]},
     {'name': '7222',
      'parent': '9433',
      'children': [{'name': '2022', 'parent': '7222'}]}]}]}]

In [89]:
# almacena diccionario como json en archivo javascript


treeData = 'var arbol = '+ json.dumps(treeData)

with open('arbol.js', 'w') as js_file:
    js_file.write(treeData)